# v1

In [56]:
# # Import packages

from datetime import datetime
from typing import Optional, Tuple, Union
from urllib.request import urlopen

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager

from loguru import logger
from bs4 import BeautifulSoup
import time


from src.Vendors.scraper import Scraper

# # STATICS
VENDOR_URL = 'https://www.ui.com/download/'
DOWNLOAD_PATH = 'downloads/'

# Selenium Webdriver Options, Download Path, Headless, Screensize, Webbrowser Version
options = Options()
options.headless = False

options.add_experimental_option("prefs", {
    "download.default_directory": rf"{DOWNLOAD_PATH}"
})

user_agent = 'foscam Download Assistant/1.0'
options.add_argument(f'user-agent={user_agent}')
# # Initialize Chrome and open Vendor Website

options.add_argument("--window-size=1920,1080")


In [57]:
# # Class UbiquitiScraper 
class UbiquitiScraper(Scraper):

    def __init__(
            self,
            logger,
            url: str = VENDOR_URL,
            headless: bool = True,
            options: Options = options,
            max_products: int = float('inf')

    ):
        self.headless = headless
        self.url = url
        self.name = "Ubiquiti"
        self.max_products = max_products
        self.driver = webdriver.Chrome(service=Service(
            ChromeDriverManager().install()), options=options)
        self.driver.implicitly_wait(0.5)  # has to be set only once
        self.logger = logger
        self.product_catalog = {}

    def _open_website(self, url: str = '') -> None:
        try:
            if not url:
                url = self.url
            self.driver.get(url)
            self.logger.debug(f'Opened {self.name} website {url}')
        except Exception as e:
            self.logger.error(f"Could not open {self.name} website {url}!")
            self.logger.error(e)

    def _close_website(self) -> None:
        self.driver.close()
        self.logger.debug('Closes Window')

    def _create_product_catalog(self) -> list:
        """parses html and returns a list of tuples with product name and product url

        Args:
            html (_type_): html of current product list

        Returns:
            list: list of tuples with product name and product url
        """

        SELECTOR_PRODUCTS = '//*[@id="downloads"]/div[2]/div[1]'
        self.driver.implicitly_wait(1)
        html = self.driver.find_element(By.XPATH, value=SELECTOR_PRODUCTS).get_attribute('innerHTML')
        product_type_list = BeautifulSoup(html, 'html.parser').find_all('li') # list of product_types
        # list of tuples (product name, product url)
        p_type_and_p_name_tuple = []
        for elem in product_type_list:
            for elem2 in elem('a'):
                try:
                    p_type_and_p_name_tuple.append((elem2.find_all('span')[-1].text, elem2['href']))
                except Exception as e:
                    print(e)
        # list of tuples (product name, product url) to product catalog dict
        for product in p_type_and_p_name_tuple:
            if product[1].startswith('#!'):
                product_type = product[1].replace('#!', '')
                self.product_catalog[product_type] = []
            else:
                self.product_catalog[product_type].append(product[1].replace('#', ''))
        # clean up product catalog, no empty lists like in training, utilities
        self.product_catalog = {k: v for k, v in self.product_catalog.items() if v}
        return self.product_catalog

    def _find_metadata_table(self, product:str) -> None:
        """ find clickable button for see all firmwares then
            find metadata table in html abd return it

        Args:
            product: products name

        Returns:
            _type_: 
        """        
        #SELECTOR_METADATA_TABLE = '/html/body/div[1]/div[1]/div[2]/div[4]/div/main/table'
        #SELECTOR_METADATA_TABLE = '/html/body/div[1]/div[1]/div[2]/div[4]/div/main/table/tbody[2]'
        SELECTOR_METADATA_TABLE = '//*[@id="downloads"]/div[2]/div[4]/div/main/table/tbody[2]'
        #SELECTOR_PAST_FW_BUTTON = '//*[@id="downloads"]/div[2]/div[4]/div/main/table/tbody[2]/tr[12]/td/a'
        SELECTOR_PAST_FW_BUTTON = '//*[@id="downloads"]/div[2]/div[4]/div/main/table/tbody[2]/tr[4]/td/a'
        try:
            self.driver.find_element(
                By.XPATH, value=SELECTOR_PAST_FW_BUTTON).click()
            self.logger.debug('Clicked "SEE PAST FIRMWARE"-button')
        except Exception as e:
            self.logger.debug('No past firmware button, searching for metadata table')
        try:
            product_metadata = self.driver.find_element(
                By.XPATH, value=SELECTOR_METADATA_TABLE).get_attribute('innerHTML')
            self.driver.implicitly_wait(1)
            firmware_list =  BeautifulSoup(product_metadata, 'html.parser').find_all('ul')
            return firmware_list
        except Exception as ex:
            self.logger.debug(f'No metadata table, skip product {product}')
            return None

    def _convert_date(self, date_str: str):
        try:
            return datetime.strptime(date_str, "%Y/%m/%d").strftime("%Y-%m-%d")
        except Exception as ex:
            self.logger.error(f'Could not convert date {date_str}')
            self.logger.error(ex)
            return None

    def scrape_metadata(self):
        """scrapes metadata from foscam website

        Returns:
            list[dict]: list of dicts with metadata
        """
        self._open_website()
        self._create_product_catalog()
        
        metadata = []
        self.logger.debug(f'Iterate over product catalog and scrape metadata')
        for product_type, product_url in self.product_catalog.items():
            if len(metadata) > self.max_products:
                    break
            try:
                self._open_website(f'{VENDOR_URL}{product_url}')
                metadata_html = self._find_metadata_table(product_type, product_url)
                fw_releases_list = BeautifulSoup(metadata_html, 'html.parser').find_all('ul')
                for fw_release in fw_releases_list:
                    metadata_current = fw_release.find_all('td')
                    tmp_metadata_dict = {'manufacturer': self.name,
                                                    'version': metadata_current[0].text,
                                                    'product_type': product_type, 
                                                    'product_name': product_url,
                                                    'url': f'{VENDOR_URL}{product_url}',
                                                    'checksum_scraped': None, # not available
                                                    'download_link': f"{VENDOR_URL}{metadata_current[-1].find_all('a')[0]['href']}",
                                                    'release_date': self._convert_date(metadata_current[1].text),
                                                    "additional_data": {} # nothing valuable
                                                    }
                                                    
                    metadata.append(tmp_metadata_dict)
                print(f'Finished scraping {product_type=}, {product_url=}')
            except Exception as ex:
                print('{product}, {product_url} is missing crucial data')
                print(ex)
                pass
            time.sleep(1)
        self.driver.quit()
        return metadata

    

In [58]:
scraper = UbiquitiScraper(logger)

In [59]:
scraper._open_website()

2023-01-14 18:13:58.272 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/


In [60]:
product_catalog = scraper._create_product_catalog()
product_catalog

list index out of range
list index out of range


{'airmax-ac': ['gigabeam',
  'isostation-ac',
  'prismstation-ac',
  'litebeam-ac',
  'litebeam-ac-gen2',
  'nanobeam-ac',
  'nanobeam-ac-gen2',
  'nanostation-ac',
  'powerbeam-ac',
  'powerbeam-ac-gen2',
  'powerbeam-ac-iso',
  'rocket-prism-ac',
  'liteap-ac',
  'rocket-ac',
  'bullet-ac',
  'airprism-sector',
  'airmax-ac-sector-antenna',
  'rocketdish-ac-antenna',
  'horn-5',
  'monsterdish-antenna',
  'gigabeam',
  'isostation-ac',
  'prismstation-ac',
  'litebeam-ac',
  'litebeam-ac-gen2',
  'nanobeam-ac',
  'nanobeam-ac-gen2',
  'nanostation-ac',
  'powerbeam-ac',
  'powerbeam-ac-gen2',
  'powerbeam-ac-iso',
  'rocket-prism-ac',
  'liteap-ac',
  'rocket-ac',
  'bullet-ac',
  'airprism-sector',
  'airmax-ac-sector-antenna',
  'rocketdish-ac-antenna',
  'horn-5',
  'monsterdish-antenna'],
 'airmax-m': ['isostation-m',
  'airgridm',
  'litebeam-m5',
  'nanobeamm',
  'powerbeam',
  'powerbeam-m5-iso',
  'bulletm',
  'nanostationm',
  'nanobridgem',
  'powerbridge-m10',
  'picostati

In [ ]:
def accept_cookies():
    """_summary_
    """
    

In [61]:
# scrape metadata
ll = []
for product_type, products in tqdm(product_catalog.items()):
    # i.e. https://www.ui.com/download/airmax-ac/powerbeam-ac
    scraper._open_website(f'{VENDOR_URL}{product_type}')
    scraper.driver.implicitly_wait(5)
        '//*[@id="reveal-eula"]'
    # scrape metadata # parse metadata
    firmware_list = scraper._find_metadata_table(product_type)
    ll.append((product_type,firmware_list))
    if firmware_list:
        scraper.logger.debug(f'Found metadata table for {product_type}, {len(firmware_list)} entries')
    else:
        scraper.logger.debug(f'No metadata table for {product_type}')
        continue

  0%|          | 0/17 [00:00<?, ?it/s]2023-01-14 18:14:01.279 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac
2023-01-14 18:14:02.760 | DEBUG    | __main__:_find_metadata_table:89 - Clicked "SEE PAST FIRMWARE"-button
2023-01-14 18:14:02.811 | DEBUG    | __main__:<module>:10 - Found metadata table for airmax-ac, 5 entries
  6%|▌         | 1/17 [00:04<01:07,  4.19s/it]2023-01-14 18:14:05.681 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-m
2023-01-14 18:14:07.363 | DEBUG    | __main__:_find_metadata_table:89 - Clicked "SEE PAST FIRMWARE"-button
2023-01-14 18:14:07.398 | DEBUG    | __main__:<module>:10 - Found metadata table for airmax-m, 3 entries
 12%|█▏        | 2/17 [00:08<01:06,  4.42s/it]2023-01-14 18:14:10.011 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-legacy
2023-01-14 18:14:11.543 | DEBUG    | __main__:_find_metadata_

In [63]:
for l in ll:
    print(l[0], len(l[1]))

airmax-ac 5
airmax-m 3
airmax-legacy 7
airfiber 10
ltu 2
ufiber 3
edgemax 11
mfi 4
sunmax 2
unifi 20
wave 2
unifi-protect 7
unifi-access 0
unifi-switching-routing 18
unifi-talk 0
unifi-led 0
accessories 9


In [90]:
ll = []
# for product type, product names
for k, values in product_catalog.items():
    # for each product / page
    for v in values:
        # scrape metadata # parse metadata
        metadata_table = scraper._find_metadata_table(k, v)
        firmware_list = BeautifulSoup(metadata_table, 'html.parser').find_all('ul')
        # for every firmware in table
        for fw in firmware_list:
            fw_dict = {}
            attributes_list = fw.get_text().split('\n\n')
            for str_elem in attributes_list:
                try:
                    fw_dict[str_elem.split(':')[0].strip()] = str_elem.split(':')[1].strip()
                except Exception as ex:
                    if 'checksum' in str_elem:
                        fw_dict['checksum'] = str_elem.split('checksum')[1].strip()
                    else:
                        print(ex)
                    pass
            #print()
        

2022-12-17 15:32:00.174 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/gigabeam
2022-12-17 15:32:00.680 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


2022-12-17 15:32:03.839 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/isostation-ac
2022-12-17 15:32:04.487 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

2022-12-17 15:32:07.633 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/prismstation-ac
2022-12-17 15:32:08.662 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

2022-12-17 15:32:12.014 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/litebeam-ac
2022-12-17 15:32:12.762 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

KeyboardInterrupt: 

2022-12-17 21:15:27.144 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/powerbeam-ac
2022-12-17 21:15:27.819 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


In [184]:
metadata_keys = ['manufacturer','product_name','product_type','version','release_date','download_link','file_path','checksum_local','checksum_scraped','emba_tested','emba_report_path','embark_report_link','additional_data']
ll = []
for product_type, products in product_catalog.items():
    for product in products:
        try:
            metadata_table = scraper._find_metadata_table(product_type, product)
            firmware_list = BeautifulSoup(metadata_table, 'html.parser').find_all('ul')
            for fw in firmware_list:
                fw_dict = dict.fromkeys(metadata_keys)
                for fw_metadata_elem in fw.find_all('li'):
                    if 'Filename' in fw_metadata_elem.get_text():
                        fw_dict['file_path'] = fw_metadata_elem.findAll('div')[-1].get_text()
                    elif 'Version' in fw_metadata_elem.get_text():
                        fw_dict['version'] = fw_metadata_elem.findAll('div')[-1].get_text()
                    elif 'Products' in fw_metadata_elem.get_text():
                        fw_dict['additional_data'] = {'products': fw_metadata_elem.findAll('div')[-1].get_text()}
                    elif 'MD5 checksum' in fw_metadata_elem.get_text():
                        fw_dict['checksum_scraped'] = fw_metadata_elem.findAll('div')[-1].get_text().replace('MD5 checksum ', '').strip()
                fw_dict['manufacturer'] = 'Ubiquiti'
                fw_dict['product_name'] = product
                fw_dict['product_type'] = product_type
                ll.append(fw_dict)
        except Exception as ex:
            print(ex)
            pass

2022-12-17 21:50:16.429 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/gigabeam
2022-12-17 21:50:16.699 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


KeyboardInterrupt: 

In [194]:
metadata_keys = ['manufacturer','product_name','product_type','version','release_date','download_link','file_path','checksum_local','checksum_scraped','emba_tested','emba_report_path','embark_report_link','additional_data']
ll = []
for product_type, products in product_catalog.items():
    for product in products[:10]:
        metadata_table = scraper._find_metadata_table(product_type, product)
        firmware_list = BeautifulSoup(metadata_table, 'html.parser').find_all('ul')
        for fw in firmware_list:
            fw_dict = dict.fromkeys(metadata_keys)
            for fw_metadata_elem in fw.find_all('li'):
                print(fw_metadata_elem)
                '//*[@id="downloads"]/div[2]/div[4]/div/main/table/tbody[2]/tr[2]/td[4]'

2022-12-17 22:53:02.123 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/gigabeam
2022-12-17 22:53:02.546 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


<li class="downloadPreview__filenameTitle">
<div class="downloadPreview__title">Filename:</div>
<div class="downloadPreview__spec">GBE.v1.4.1.1bf05681.221028.1153.bin</div>
</li>
<li class="downloadPreview__type">
<div class="downloadPreview__title">Type:</div>
<div class="downloadPreview__spec"><span class="icon ubnt-icon--firmware-update downloadResults__typeIcon"></span>Firmware</div>
</li>
<li>
<div class="downloadPreview__title">Version:</div>
<div class="downloadPreview__spec">v1.4.1</div>
</li>
<li>
<div class="downloadPreview__title">Size:</div>
<div class="downloadPreview__spec">11.57MB</div>
</li>
<li class="downloadPreview__otherProducts">
<div class="downloadPreview__title"> Products:</div>
<div class="downloadPreview__spec">GBE</div>
</li>
<li class="downloadPreview__description">
<div class="downloadPreview__title">Description:</div>
<div class="downloadPreview__spec">
                            MD5 checksum 1b67fd49247d811cede8aeff7bf2b061
                        </div>

2022-12-17 22:53:05.394 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/isostation-ac
2022-12-17 22:53:06.430 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


<li class="downloadPreview__filenameTitle">
<div class="downloadPreview__title">Filename:</div>
<div class="downloadPreview__spec">WA.v8.7.11.46972.220614.0420.bin</div>
</li>
<li class="downloadPreview__type">
<div class="downloadPreview__title">Type:</div>
<div class="downloadPreview__spec"><span class="icon ubnt-icon--firmware-update downloadResults__typeIcon"></span>Firmware</div>
</li>
<li>
<div class="downloadPreview__title">Version:</div>
<div class="downloadPreview__spec">v8.7.11</div>
</li>
<li>
<div class="downloadPreview__title">Size:</div>
<div class="downloadPreview__spec">9.7MB</div>
</li>
<li class="downloadPreview__otherProducts">
<div class="downloadPreview__title"> Products:</div>
<div class="downloadPreview__spec">IS-5AC, LAP-120, LAP-GPS, LBE-5AC-23, LBE-5AC-Gen2, LBE-5AC-LR, Loco5AC, NBE-5AC-16, NBE-5AC-Gen2, NS-5AC, PBE-5AC-300, PBE-5AC-300-ISO, PBE-5AC-400, PBE-5AC-400-ISO, PBE-5AC-Gen2</div>
</li>
<li class="downloadPreview__description">
<div class="downloadPre

2022-12-17 22:53:09.307 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/prismstation-ac
2022-12-17 22:53:09.963 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


...

2022-12-17 22:53:30.640 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/powerbeam-ac
2022-12-17 22:53:31.293 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


<li class="downloadPreview__filenameTitle">
<div class="downloadPreview__title">Filename:</div>
<div class="downloadPreview__spec">WA.v8.7.11.46972.220614.0420.bin</div>
</li>
<li class="downloadPreview__type">
<div class="downloadPreview__title">Type:</div>
<div class="downloadPreview__spec"><span class="icon ubnt-icon--firmware-update downloadResults__typeIcon"></span>Firmware</div>
</li>
<li>
<div class="downloadPreview__title">Version:</div>
<div class="downloadPreview__spec">v8.7.11</div>
</li>
<li>
<div class="downloadPreview__title">Size:</div>
<div class="downloadPreview__spec">9.7MB</div>
</li>
<li class="downloadPreview__otherProducts">
<div class="downloadPreview__title"> Products:</div>
<div class="downloadPreview__spec">IS-5AC, LAP-120, LAP-GPS, LBE-5AC-23, LBE-5AC-Gen2, LBE-5AC-LR, Loco5AC, NBE-5AC-16, NBE-5AC-Gen2, NS-5AC, PBE-5AC-300, PBE-5AC-300-ISO, PBE-5AC-400, PBE-5AC-400-ISO, PBE-5AC-Gen2</div>
</li>
<li class="downloadPreview__description">
<div class="downloadPre

2022-12-17 22:53:34.056 | DEBUG    | __main__:_open_website:28 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/powerbeam-ac-gen2
2022-12-17 22:53:34.693 | DEBUG    | __main__:_find_metadata_table:90 - Clicked See past firmware button


<li class="downloadPreview__filenameTitle">
<div class="downloadPreview__title">Filename:</div>
<div class="downloadPreview__spec">2WA.v8.7.11.46972.220614.0419.bin</div>
</li>
<li class="downloadPreview__type">
<div class="downloadPreview__title">Type:</div>
<div class="downloadPreview__spec"><span class="icon ubnt-icon--firmware-update downloadResults__typeIcon"></span>Firmware</div>
</li>
<li>
<div class="downloadPreview__title">Version:</div>
<div class="downloadPreview__spec">v8.7.11</div>
</li>
<li>
<div class="downloadPreview__title">Size:</div>
<div class="downloadPreview__spec">9.7MB</div>
</li>
<li class="downloadPreview__otherProducts">
<div class="downloadPreview__title"> Products:</div>
<div class="downloadPreview__spec">B-DB-AC, BulletAC-IP67, NBE-2AC-13, PBE-2AC-400</div>
</li>
<li class="downloadPreview__description">
<div class="downloadPreview__title">Description:</div>
<div class="downloadPreview__spec">
                            MD5 checksum a3b2cc26223804c42ec9ce

KeyboardInterrupt: 

In [ ]:
            """
            fw_attributes = fw.get_text().split(':')
            fw_attributes = [x.strip().replace('\n','') for x in fw_attributes]
            print(fw_attributes)
            print(fw_attributes[1].replace('Type', ''))
            print(fw_attributes[3].replace('Size', '')) 
            print(fw_attributes[6].replace('MD5 checksum ', ''))
            print('***')"""

In [ ]:
ll

[{'Filename': 'XC.v8.7.11.46972.220614.0419.bin',
  'Type': 'Firmware',
  'Version': 'v8.7.11',
  'Size': '9.8MB',
  'Products': 'NBE-5AC-19, NBE-5AC-Gen2, PBE-5AC-500, PBE-5AC-500-ISO, PBE-5AC-620, PS-5AC, R5AC-Lite, R5-AC-PRISM, R5AC-PTMP, R5AC-PTP, RP-5AC-Gen2',
  'Description': ''},
 {'Filename': 'XC.v8.7.11.46972.220614.0419.bin',
  'Type': 'Firmware',
  'Version': 'v8.7.11',
  'Size': '9.8MB',
  'Products': 'NBE-5AC-19, NBE-5AC-Gen2, PBE-5AC-500, PBE-5AC-500-ISO, PBE-5AC-620, PS-5AC, R5AC-Lite, R5-AC-PRISM, R5AC-PTMP, R5AC-PTP, RP-5AC-Gen2',
  'Description': ''},
 {'Filename': 'XC.v8.7.11.46972.220614.0419.bin',
  'Type': 'Firmware',
  'Version': 'v8.7.11',
  'Size': '9.8MB',
  'Products': 'NBE-5AC-19, NBE-5AC-Gen2, PBE-5AC-500, PBE-5AC-500-ISO, PBE-5AC-620, PS-5AC, R5AC-Lite, R5-AC-PRISM, R5AC-PTMP, R5AC-PTP, RP-5AC-Gen2',
  'Description': ''},
 {'Filename': 'XC.v8.7.11.46972.220614.0419.bin',
  'Type': 'Firmware',
  'Version': 'v8.7.11',
  'Size': '9.8MB',
  'Products': 'NBE-5A

GBE.v1.4.1.1bf05681.221028.1153.bin
v1.4.1
1b67fd49247d811cede8aeff7bf2b061
***
GBE.v1.4.0.1e71455a.210726.1326.bin
v1.4.0
47c9ce60f95f7018297c76af75ac3056
***
GBE.v1.3.0.492789c1.210406.0813.bin
v1.3.0
81f8dc779f424512c46fe362ce26be57
***
GBE.v1.2.0.ec5ea39c.201015.1407.bin
v1.2.0
730066aeed53dab323f090973f5cb7a6
***
GBE.v0.11.1-cs.f16f621c.200826.1546.bin
v0.11.1
99d7216db91e4c54da426d28d4cf43d0
***


In [109]:
import BeautifiulSoup

ModuleNotFoundError: No module named 'BeautifiulSoup'

In [ ]:
def get_product_type_list(self) -> list:
        """parses html and returns a list of tuples with product name and product url

        Args:
            html (_type_): html of current product list

        Returns:
            list: list of tuples with product name and product url
        """    
        SELECTOR_PRODUCT_TYPE = '//*[@id="downloads"]/div[1]'
        self.driver.implicitly_wait(1)
        html = self.driver.find_element(By.XPATH, value=SELECTOR_PRODUCT_TYPE).get_attribute('innerHTML')
        product_list = BeautifulSoup(html, 'html.parser').find_all('li')
        products_tuple_list = [] # list of tuples with product name and product url
        for product in product_list:
            product_name = product.find_all('a')[0]['data-title']
            # append product name and product url
            products_tuple_list.append((product_name, f"{VENDOR_URL}{product_name.lower().replace(' ', '-')}"))
        
        return products_tuple_list

In [ ]:
def _create_product_catalog(self) -> list:
        """parses html and returns a list of tuples with product name and product url

        Args:
            html (_type_): html of current product list

        Returns:
            list: list of tuples with product name and product url
        """

        SELECTOR_PRODUCTS = '//*[@id="downloads"]/div[2]/div[1]'
        self.driver.implicitly_wait(1)
        html = self.driver.find_element(By.XPATH, value=SELECTOR_PRODUCTS).get_attribute('innerHTML')
        product_type_list = BeautifulSoup(html, 'html.parser').find_all('li') # list of product_types
        # list of tuples (product name, product url)
        p_type_and_p_name_tuple = []
        for elem in product_type_list:
            for elem2 in elem('a'):
                try:
                    p_type_and_p_name_tuple.append((elem2.find_all('span')[-1].text, elem2['href']))
                except Exception as e:
                    print(e)
        # list of tuples (product name, product url) to product catalog dict
        for product in p_type_and_p_name_tuple:
            if product[1].startswith('#!'):
                product_type = product[1].replace('#!', '')
                self.product_catalog[product_type] = []
            else:
                self.product_catalog[product_type].append(product[1].replace('#', ''))
        # clean up product catalog, no empty lists like in training, utilities
        self.product_catalog = {k: v for k, v in self.product_catalog.items() if v}
        return self.product_catalog

In [ ]:
product_catalog = create_product_catalog(scraper)

list index out of range
list index out of range


In [ ]:
def find_metadata_table(self, product_type:str, product_url: str):
    """ find clickable button for see all firmwares then
        find metadata table in html abd return it

    Args:
        product_type (str): product type
        product_url (str): product url / product name

    Returns:
        _type_: 
    """        
    # i.e. https://www.ui.com/download/airmax-ac/powerbeam-ac
    self._open_website(f'{VENDOR_URL}{product_type}/{product_url}')
    #SELECTOR_METADATA_TABLE = '/html/body/div[1]/div[1]/div[2]/div[4]/div/main/table'
    SELECTOR_METADATA_TABLE = '/html/body/div[1]/div[1]/div[2]/div[4]/div/main/table/tbody[2]'
    SELECTOR_PAST_FW_BUTTON = '//*[@id="downloads"]/div[2]/div[4]/div/main/table/tbody[2]/tr[4]/td/a'
    self.driver.implicitly_wait(1)
    try:
        self.driver.find_element(
            By.XPATH, value=SELECTOR_PAST_FW_BUTTON).click()
    except Exception as e:
        self.logger.debug('No past firmware button, searching for metadata table')
    try:
        product_metadata = self.driver.find_element(
            By.XPATH, value=SELECTOR_METADATA_TABLE).get_attribute('innerHTML')
        self.driver.implicitly_wait(1)
        return product_metadata
    except Exception as e:
        self.logger.debug('No metadata table, skip product')
        return None

In [ ]:
# get first key from dicrionary
list(product_catalog.keys())[0], product_catalog[list(product_catalog.keys())[0]][0]

('airmax-ac', 'gigabeam')

In [ ]:
table = find_metadata_table(scraper, list(product_catalog.keys())[0], product_catalog[list(product_catalog.keys())[0]][1])

2022-12-16 11:26:22.799 | DEBUG    | __main__:_open_website:27 - Opened Ubiquiti website https://www.ui.com/download/airmax-ac/isostation-ac


In [ ]:
def scrape_metadata(self):
    """scrapes metadata from foscam website

    Returns:
        list[dict]: list of dicts with metadata
    """
    self._open_website()
    self._create_product_catalog()

    metadata = []
    self.logger.debug(f'Iterate over product catalog and scrape metadata')
    for product, product_url in self.products_list:
        if len(metadata) > self.max_products:
            break
        try:
            self._open_website(f'{VENDOR_URL}{product_url}')
            metadata_html = self._find_metadata_table(product_url)
            fw_releases_list = BeautifulSoup(
                metadata_html, 'html.parser').find_all('tr')
            for fw_release in fw_releases_list[1:]:
                metadata_current = fw_release.find_all('td')
                tmp_metadata_dict = {'manufacturer': 'foscam',
                                        'version': metadata_current[0].text,
                                        'product_type': None,  # not available
                                        'product_name': product,
                                        'url': f'{VENDOR_URL}{product_url}',
                                        'checksum_scraped': None,  # not available
                                        'download_link': f"{VENDOR_URL}{metadata_current[-1].find_all('a')[0]['href']}",
                                        'release_date': self._convert_date(metadata_current[1].text),
                                        "additional_data": {}  # nothing valuable
                                        }

                metadata.append(tmp_metadata_dict)
            print(f'Finished scraping {product=}, {product_url=}')
        except Exception as ex:
            self.logger.debug(
                f'{product=}, {product_url=} is missing crucial data')
            self.logger.debug(ex)
            pass
        time.sleep(1)
    self.driver.quit()
    return metadata

In [ ]:
for firmware_elem in BeautifulSoup (table, 'html.parser').find_all('li'):
    for tag in firmware_elem:
        print(tag.text)
    print('***')



Filename:


WA.v8.7.11.46972.220614.0420.bin


***


Type:


Firmware


***


Version:


v8.7.11


***


Size:


9.7MB


***


 Products:


IS-5AC, LAP-120, LAP-GPS, LBE-5AC-23, LBE-5AC-Gen2, LBE-5AC-LR, Loco5AC, NBE-5AC-16, NBE-5AC-Gen2, NS-5AC, PBE-5AC-300, PBE-5AC-300-ISO, PBE-5AC-400, PBE-5AC-400-ISO, PBE-5AC-Gen2


***


Description:



                            MD5 checksum 855887a0a61c93e05c9448b2a3aca4d2
                        


***


Filename:


WA.v8.7.9.46843.220329.1728.bin


***


Type:


Firmware


***


Version:


v8.7.9


***


Size:


9.7MB


***


 Products:


IS-5AC, LAP-120, LAP-GPS, LBE-5AC-23, LBE-5AC-Gen2, LBE-5AC-LR, Loco5AC, NBE-5AC-16, NBE-5AC-Gen2, NS-5AC, PBE-5AC-300, PBE-5AC-300-ISO, PBE-5AC-400, PBE-5AC-400-ISO, PBE-5AC-Gen2


***


Description:



                            MD5 checksum 8f855b17ee0995c67dc323b3394e631b
                        


***


Filename:


WA.v8.7.8.46705.220201.1819.bin


***


Type:


Firmware


***


Version:


v8.7.8


*

In [ ]:
if __name__ == '__main__':
    from loguru import logger
    import json 
    logger.debug('Start Ubiquiti')
    
    metadata = scraper.scrape_metadata()
    
    # save metadata to json file
    with open("scraped_metadata/firmware_data_foscam.json", "w") as firmware_file:
        json.dump(metadata, firmware_file)

    scraper.logger.debug(f'Finished {scraper.name}')


2022-12-11 19:52:11.767 | DEBUG    | __main__:<module>:4 - Start foscam
2022-12-11 19:53:52.104 | DEBUG    | __main__:_open_website:26 - Opened foscam website https://www.foscam.com/downloads/index.html
2022-12-11 19:53:52.108 | DEBUG    | __main__:_create_product_catalog:86 - get html of product caroussel
2022-12-11 19:53:52.109 | DEBUG    | __main__:_create_product_catalog:89 - Iterare over pages to create product catalog
2022-12-11 19:54:07.727 | DEBUG    | __main__:_next_page:73 - No next page
2022-12-11 19:54:07.728 | DEBUG    | __main__:_next_page:74 - Found counter=7 pages
2022-12-11 19:54:07.729 | DEBUG    | __main__:scrape_metadata:136 - Iterate over product catalog and scrape metadata
2022-12-11 19:54:08.324 | DEBUG    | __main__:_open_website:26 - Opened foscam website https://www.foscam.com/downloads/firmware_details.html?id=161


2021/12/08
2022/10/26
Finished scraping product='VD1/DBW5', product_url='/downloads/firmware_details.html?id=161'


2022-12-11 19:54:09.791 | DEBUG    | __main__:_open_website:26 - Opened foscam website https://www.foscam.com/downloads/firmware_details.html?id=163


2021/12/08
2022/10/26
Finished scraping product='F41/FLC', product_url='/downloads/firmware_details.html?id=163'


2022-12-11 19:54:11.231 | DEBUG    | __main__:_open_website:26 - Opened foscam website https://www.foscam.com/downloads/firmware_details.html?id=162


...

2022-12-11 19:54:55.997 | DEBUG    | __main__:_open_website:26 - Opened foscam website https://www.foscam.com/downloads/firmware_details.html?id=16


2019/07/25
2018/11/19
2018/5/11
2017/6/14
Finished scraping product='FI9803EP', product_url='/downloads/firmware_details.html?id=16'


2022-12-11 19:54:57.416 | DEBUG    | __main__:_open_website:26 - Opened foscam website https://www.foscam.com/downloads/firmware_details.html?id=120


2017/08/30
Finished scraping product='FI9800E', product_url='/downloads/firmware_details.html?id=120'


KeyboardInterrupt: 

In [82]:
ll

[]